# 16S profiling analysis pipeline (Illumina paired-end) acc brmicrobiome with MO modifications

**BMP** advisory board recommend the use of this pipeline as a standard for 16S rRNA data analysis.  
We are now working in order to improve this workflow besides making it easier for end-users.   
If you have any questions or suggestions, please contact  
  
**Victor Pylro**: victor.pylro@brmicrobiome.org, Leandro Lemos: lemosbioinfo@gmail.com or Luiz Roesch: luiz.roesch@brmicrobiome.org
  
Please, cite our efforts when using this pipeline: Data analysis for 16S microbial profiling from different benchtop sequencing platforms. J Microbiol Methods. 2014. doi: 10.1016/j.mimet.2014.08.018.  
  
  
The authors are grateful to Dr Victor Pylro for the advice in modification of original vsearch 16S (Illumina paired-end) bmp pipeline.  
  
Sequences in this pipeline are without barcodes - already demultiplexed to the samples (separate folders with sample name). For that reason Steps 1. - 5. of this pipeline are different from bmp, but the same as ITS pipeline acc Balint et al 2014 modified by Oskiera et al 2018.  
  
Please, cite our efforts when using this pipeline: Ballint et al 2014 "An Illumina metabarcoding pipeline for fungi. Ecology and Evolution 4 (13) 2642–2653. DOI: 10.1002/ece3.1107  
  
**Please cite also: Oskiera et al. 2018**

## Steps 1. - 5. acc Balint  et al 
  
## 1. Quality filtering
**FILTER PARAMETERS: -sc 33 -q 20 -l 100 -ld N**
  
```perl Reads_Quality_Length_distribution.pl -fw forw_reads.fastq -rw rev_reads.fastq -sc 33 -q 20 -l 100 -ld N  ```  

In [ ]:
%%bash
for d in ./data/*
do
    ( cd $d && perl ../../bin/Reads_Quality_Length_distribution.pl -fw ./*_R1.fastq -rw ./*_R2.fastq -sc 33 -q 20 -l 100 -ld N )
done

## 2. Paired-end assembly acc Balint et all with MO modification (PEAR)  
  
  
```pear -f Filtered_*_R1.fastq -r Filtered_*_R2.fastq -m 400 -n 50 -v 25 -o pear_paired_assembled.fastq```
  
**PEAR settings:**   
Maximum assembly length............: 400  
Minimum assembly length............: 50  
Minimum overlap....................: 25   

In [ ]:
%%bash
for d in ./data/*
do
    ( cd $d && mkdir ./pear && pear -f Filtered_*_R1.fastq -r Filtered_*_R2.fastq -m 400 -n 50 -v 25 -o ./pear/paired_assembled.fastq )
done

## 3. Remove primer artifacts  
**Run code:**  
  
```python remove_multiprimer.py -i input.fastq -o output.fastq -f '<'forPrimerSeq'>' -r '<'revPrimerSeq'>'  ```
  
**```python remove_multiprimer.py```**  
-i pear_paired_assembled.fastq.assembled.fastq  
-o pear_paired_assembled_output_multiprimer.fastq  
Primers:  
-f GTGYCAGCMGCCGCGGTAA = GTG[CT]CAGC[AC]GCCGCGGTAA   
-r GGACTACNVGGGTWTCTAAT = GGACTAC[ACGT][ACG]GGGT[AT]TCTAAT  

In [ ]:
%%bash
for d in ./data/*
do
    ( cd $d && python2 ../../bin/remove_multiprimer.py -i ./pear/paired_assembled.fastq.assembled.fastq -o ./pear/paired_assembled_output_multiprimer.fastq -f GTG[CT]CAGC[AC]GCCGCGGTAA -r GGACTAC[ACGT][ACG]GGGT[AT]TCTAAT )
done

## 4. Reorient reads to 5'-3'  
**Run code:**  
  
  
```fqgrep -m2 -p GTG[CT]CAGC[AC]GCCGCGGTAA -e pear_paired_assembled_output_multiprimer.fastq > good_5-3.fastq```  
  
  
```fqgrep -m2 -p GGACTAC[ACGT][ACG]GGGT[AT]TCTAAT -e pear_paired_assembled_output_multiprimer.fastq > good_3-5.fastq```  

In [ ]:
%%bash
for d in ./data/*
do
    ( cd $d && fqgrep -m 2 -p GTG[CT]CAGC[AC]GCCGCGGTAA -e ./pear/paired_assembled_output_multiprimer.fastq > ./pear/good_5-3.fastq )
done

In [ ]:
%%bash
for d in ./data/*
do
    ( cd $d && fqgrep -m 2 -p ATTAGA[AT]ACCC[CGT][AGT]GTAGTCC -e ./pear/paired_assembled_output_multiprimer.fastq > ./pear/good_3-5.fastq )
done

In [ ]:
%%bash
for d in ./data/*
do
    ( cd $d && fastx_reverse_complement -Q 33 -i ./pear/good_3-5.fastq >> ./pear/good_5-3.fastq )
done

## 5. Demultiplexing
  
**Run code:**  
  
```bash demultiplex.sh forward_labels.csv reverse_labels.csv 5-3_oriented.fastq```

In [ ]:
%%bash
for d in ./data/*
do
    ( cd $d && bash ../../bin/demultiplex.sh ../../bin/forward_labels.csv ../../bin/reverse_labels.csv ./pear/good_5-3.fastq )
done

## Steps from 6. to the end of this pipeline acc 16S profiling analysis pipeline (Illumina paired-end) - BMP advisory. 

## 6. Quality filtering, length truncate, and convert to FASTA  ```<<<USING VSEARCH>>>```
  
```vsearch --fastx_filter $PWD/slout/seqs.fastq --fastq_maxee 1.0 --fastq_trunclen 240 --fastaout reads.fa```  
  
The -fastq_trunclen parameter must be adjusted as your needs. You can change  
  ** ```-fastq_maxee``` **   
  
parameter if you want. I recommend to keep it as **```1.0```**.  

In [ ]:
%%bash
for d in ./data/*
do
    ( cd $d && vsearch --fastx_filter ./pear/good_5-3.fastq --fastq_maxee 1.0 --fastq_trunclen 240 -fastaout ./pear/reads.fasta )
done

## 7. Modification acc Dr. Victor Pylro Brazilian Microbiome Project  
  
http://brmicrobiome.org  
victor.pyro@brmicrobiome.org  
"We need to change the header of your reads to make them compatible with UPARSE pipeline. For this, we wrote a script  

** ```bmp_demultiplexed``` ** - ```./bin/bmp_demultiplexed.pl```  
  
Basically, it changes your headers to  
  
 ```  
">Samplename_X;barcodelabel=Samplename”
```   "

In [ ]:
%%bash
for d in ./data/*
do
       ( cd $d && CURRENT=`pwd` && BASENAME=`basename "$CURRENT"` && echo "$BASENAME" && perl ../../bin/bmp_demultiplexed.pl -i ./pear/reads.fasta -o ./demultiplexed.fasta -b "$BASENAME" ) 
done

## 8. Dereplication ```<<<USING VSEARCH>>>```  
  
``` vsearch --derep_fulllength $PWD/reads_uparse.fa --output derep.fa --sizeout```  

In [ ]:
%%bash
for d in ./data/*
do
    ( cd $d && vsearch \
    --derep_fulllength ./demultiplexed.fasta \
    --sizeout \
    --fasta_width 0 \
    --output ./dereplicated.fasta )
done

## 9. Merge all resulting files using a **```“cat”``` command**.   

In [ ]:
%%bash
for d in ./data/*
do
    ( cd $d && cat ./dereplicated.fasta >> ../../all_samples.fasta )
done

## 10. Abundance sort and discard singletons ```<<<USING VSEARCH>>>```    
  
```vsearch --sortbysize $PWD/derep.fa --output sorted.fa --minsize 2```  

In [ ]:
!vsearch \
    --sortbysize ./all_samples.fasta \
    --minsize 2 \
    --output ./sorted.fasta

  
## 11. OTU clustering using UPARSE method ```<<<USING VSEARCH>>>```  
  
```vsearch --cluster_size $PWD/sorted.fa --consout otus1.fa --id 0.97```

In [ ]:
! ./bin/vsearch --cluster_size ./sorted.fasta --consout ./otus97.fasta --id 0.97

In [ ]:
! ./bin/vsearch --cluster_size ./sorted.fasta --consout ./otus99.fasta --id 0.99

## 12. Fasta Formatter ```<<<FASTX TOOLKIT SCRIPT>>>```  
  
```fasta_formatter -i otus1.fa -o formated_otus1.fa```

In [ ]:
!fasta_formatter -i ./otus97.fasta -o ./formated_otus97.fasta

In [ ]:
!fasta_formatter -i ./otus99.fasta -o ./formated_otus99.fasta

## 13. Renamer ```<<<BMP SCRIPT>>>```  
  
```perl bmp-otuName.pl -i formated_otus1.fa -o otus.fa```  

In [ ]:
!perl ./bin/bmp-otuName.pl -i ./formated_otus97.fasta -o ./otus.fasta

In [ ]:
!perl ./bin/bmp-otuName.pl -i ./formated_otus99.fasta -o ./otus99.fasta

## 14. Map reads back to OTU database ``` <<<VSEARCH>>> ```   
  
  
```vsearch --usearch_global $PWD/reads_uparse.fa --db otus.fa --strand plus --id 0.97 --uc map.uc```  
  
  

In [ ]:
!vsearch \
        --usearch_global ./sorted.fasta \
        --db ./otus.fasta \
        --strand plus \
        --id 0.97 \
        --uc ./map_97.uc

In [ ]:
!vsearch \
        --usearch_global ./sorted.fasta \
        --db ./otus99.fasta \
        --strand plus \
        --id 0.99 \
        --uc ./map_99.uc

## 15. Assign taxonomy to OTUS using uclust method on QIIME (use the file “otus.fa” from UPARSE as input file)
  
```assign_taxonomy.py -i $PWD/otus.fa -o output```  
  

In [ ]:
!assign_taxonomy.py -i ./otus.fasta -o ./output

In [ ]:
!assign_taxonomy.py -i ./otus99.fasta -o ./output99

  
## 16. Align sequences on QIIME, using greengenes reference sequences (use the file “otus.fa” from UPARSE as input file)  

```align_seqs.py -i $PWD/otus.fa -o rep_set_align```  

  

In [ ]:
!align_seqs.py -i ./otus.fasta -o ./rep_set_align

In [ ]:
!align_seqs.py -i ./otus99.fasta -o ./rep_set_align99

## 17. Filter alignments on QIIME
  
```filter_alignment.py -i $PWD/otus_aligned.fasta -o filtered_alignment``` 

In [ ]:
!filter_alignment.py -i ./rep_set_align/otus_aligned.fasta -o ./filtered_alignment

In [ ]:
!filter_alignment.py -i ./rep_set_align99/otus99_aligned.fasta -o ./filtered_alignment99

## 18. Make the reference tree on QIIME
  
```make_phylogeny.py -i $PWD/otus_aligned_pfiltered.fasta -o rep_set.tre```

In [ ]:
!make_phylogeny.py -i ./filtered_alignment/otus_aligned_pfiltered.fasta -o ./rep_set.tre

In [ ]:
!make_phylogeny.py -i ./filtered_alignment99/otus99_aligned_pfiltered.fasta -o ./rep_set99.tre

## 19. Convert UC to otu-table.txt ```<<< BMP SCRIPT>>>```  
  
```bmp-map2qiime.py map.uc > otu_table.txt```

In [ ]:
!python2 ./bin/bmp-map2qiime.py map_97.uc > otu_table_97.txt

In [ ]:
!python2 ./bin/bmp-map2qiime.py map_99.uc > otu_table_99.txt

## 20. Convert otu_table.txt to otu-table.biom ```<<< QIIME SCRIPT>>>```  
  
```make_otu_table.py -i otu_table.txt -t otus_tax_assignments.txt -o otu_table.biom```  

In [ ]:
!make_otu_table.py -i ./otu_table_97.txt -t ./output/otus_tax_assignments.txt -o ./otu_table.biom

In [ ]:
!make_otu_table.py -i ./otu_table_99.txt -t ./output99/otus99_tax_assignments.txt -o ./otu_table99.biom

## 21. Check OTU Table  on QIIME.
  
```biom summarize-table -i $PWD/otu_table.biom -o results_biom_table```  

In [ ]:
!biom summarize-table -i ./otu_table.biom -o ./results_biom_table

In [ ]:
!biom summarize-table -i ./otu_table99.biom -o ./results_biom_table99

The summary file contains information about the number of sequences per sample, which will help us to make decisions about rarefaction (subsampling). When we inspect the file, we see that sample F3D142.S208 has 2212 reads, the minimum observed. This is what we will use as a subsampling depth. Also, a lot of the info in this file is typically reported in methods sections of manuscripts.

## Set UP ```-e``` parameter:

In [ ]:
!grep Min: ./results_biom_table && head ./results_biom_table

In [ ]:
!grep Min: ./results_biom_table99 && head ./results_biom_table99

## 22. Run diversity analyses on QIIME (or any other analysis of your choice).  
  
The parameter **```“-e”```** is the sequencing depth to use for even sub-sampling and maximum rarefaction depth. You should review the output of the ```biom summarize-table``` (step 21) command to decide on this value.  
  
```core_diversity_analyses.py -i $PWD/otu_table.biom -m $PWD/mapping_file.txt -t $PWD/rep_set.tre -e xxxx -o $PWD/core_output```  
   

```core_diversity_analyses.py -i otu_table.biom -m otu_table.txt -t rep_set.tre -e 9356 -o ./core_output```  
-i - input file otu_table.biom from 20. ```make_otu_table.py```  
-m - mapping file  
-t - tree file rep_set.tre z pkt 18. ```make_phylogeny.py```  
-e - min Count/Sample Summary from ```results_biom_table``` file  
-o - output folder localisation    

In [ ]:
!validate_mapping_file.py -m ./bin/map.txt -o validate_map -p -b -j run_prefix

In [ ]:
!core_diversity_analyses.py -i ./otu_table.biom -m ./bin/map.txt -t ./rep_set.tre -e 30563 -o ./core_output_97

In [ ]:
!core_diversity_analyses.py -i ./otu_table99.biom -m ./bin/map.txt -t ./rep_set99.tre -e 30533 -o ./core_output_99

In [ ]:
!biom add-metadata -i ./otu_table.biom -o otu_table_metadata.biom -m map.txt

In [ ]:
!biom add-metadata -i ./otu_table99.biom -o otu_table_metadata99.biom -m map.txt

In [ ]:
!biom convert -i otu_table_metadata.biom -o otu_table_json.biom --table-type="OTU table" --to-json

In [ ]:
!biom convert -i otu_table_metadata99.biom -o otu_table_json99.biom --table-type="OTU table" --to-json